# Inferential statistics
## Part III - Inferential Analysis

We're now going to look for answers to the ongoing basketball discussions between you and your family. The main ones we want to reasearch are the following:

- Your grandmother says that your sister couldn't play in a professional basketball league (not only the WNBA, but ANY professional basketball league) because she's too skinny and lacks muscle.
- Your sister says that most female professional players fail their free throws.
- Your brother-in-law heard on the TV that the average assists among NBA (male) and WNBA (female) players is 52 for the 2016-2017 season. He is convinced this average would be higher if we only considered the players from the WNBA.

Let's investigate these claims and see if we can find proof to refute or support them.

### Libraries
Import the necessary libraries first.

In [13]:
# Libraries
import math
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from scipy.stats import ttest_1samp
pd.set_option('display.max_columns', 50)

import scipy.stats as st

### Load the dataset

Load the cleaned dataset.

In [40]:
#your code here

wnba = pd.read_csv("../data/wnba_clean.csv")

print(wnba.shape)

wnba.head()

(142, 32)


Name Team  Pos  Height  Weight        BMI Birth_Place  \
0    Aerial Powers  DAL    F     183      71  21.200991          US   
1      Alana Beard   LA  G/F     185      73  21.329438          US   
2     Alex Bentley  CON    G     170      69  23.875433          US   
3  Alex Montgomery  SAN  G/F     185      84  24.543462          US   
4     Alexis Jones  MIN    G     175      78  25.469388          US   

           Birthdate  Age         College Experience  Games Played  MIN  FGM  \
0   January 17, 1994   23  Michigan State          2             8  173   30   
1       May 14, 1982   35            Duke         12            30  947   90   
2   October 27, 1990   26      Penn State          4            26  617   82   
3  December 11, 1988   28    Georgia Tech          6            31  721   75   
4     August 5, 1994   23          Baylor          R            24  137   16   

   FGA   FG%  3PM  3PA   3P%  FTM  FTA   FT%  OREB  DREB  REB  AST  STL  BLK  \
0   85  35.3   12   32  37.5   21   26  80.8     6    22   28   12    3    6   
1  177  50.8    5   18  27.8   32   41  78.0    19    82  101   72   63   13   
2  218  37.6   19   64  29.7   35   42  83.3     4    36   40   78   22    3   
3  195  38.5   21   68  30.9   17   21  81.0    35   134  169   65   20   10   
4   50  32.0    7   20  35.0   11   12  91.7     3     9   12   12    7    0   

   TO  PTS  DD2  TD3  
0  12   93    0    0  
1  40  217    0    0  
2  24  218    0    0  
3  38  188    2    0  
4  14   50    0    0

# Question 1: Can my sister play in a professional female basketball league?

As we said, you grandmother is convinced that your sister couldn't play in a professional league because of her physique and weight (her weight is 67kg). 

To find an actual answer to the question we first need to know what's the average weight of a professional female basketball player. The data we have only refers to the WNBA league and not to every female professional basketball league in the world, therefore we have no way of actually calculating it.

Still, given that we do have *some* data we can **infer** it using a sample of players like the one we have. 

**How would you do it? Try and think about the requirements that your sample must satisfy in order to be used to infer the average weight. Do you feel it actually fulfills those requirements? Do you need to make any assumptions? We could calculate a confidence interval to do the inference, but do you know any other ways?**

In [7]:
# your answer here

# from our plots in exploratory data analysis we saw that weight of wnba players follows a rather
# normal distribution. therefore it would be possible to use this sample and normal distribution in
# order to inger from the sample to the population (women basketball players)

# for a more accurate outcome, we could consider removing weight outliers -> see previous step:

# Checking top/bottom 5 comparing weight to height and age to birthdate:
# 104-113 kg for 185-196 cm, 55-59 kg for 165-175 cm 

# Drop outliers based on weight column
wnba = wnba[(wnba["Weight"] >= 59) & (wnba["Weight"] <= 104)]

print(wnba.shape)

(136, 32)


**Now that all the requirements have been taken into account, compute the confidence interval of the average weight with a confidence level of 95%.**

In [18]:
# your code here

sample_mean = wnba["Weight"].mean()
sample_std = wnba["Weight"].std(ddof = 1)
sample_n = len(wnba["Weight"])
alpha = 0.95


ci = st.norm.interval(confidence = alpha, loc = sample_mean, scale = sample_std / np.sqrt(sample_n))

print(sample_mean)
print(sample_std)
print(sample_n)
print(alpha)
print(ci)

78.75735294117646
9.578909180555195
136
0.95
(77.14746853948338, 80.36723734286954)


**What can you say about these results?**

In [10]:
#your-answer-here

# with 95% confidence, the mean weight of a female basketball player is between 77.1 and 80.4 kg

**If your sister weighs 67kg what would you tell your grandmother in regards to her assumption?**

In [9]:
#your-answer-here

# my sister's weight of 67 kg is quite below the ci lower limit as calculated above (67 < 77.1 kg)
# i would tell my grandmother that her assumption is most probably right

## Bonus: Can you plot the probability distribution of the average weight, indicating where the critical region is?

In [8]:
# your code here

# Question 2: Do female professional basketball players fail the majority of their free throws?

You do not agree with your sister when she says that most female players fail their free throws. You decide to try and estimate the percentage of players that fail more than 40% of their free throws using, you guessed it, the WNBA sample.

**How would you do it? Try and think about the requirements that your sample must satisfy in order to be used to infer the proportion of players that miss more than 40% of their free throws. Do you feel it actually fulfills those requirements? Do you need to make any assumptions?**

In [26]:
# your answer here

# i need to reload my dataset as I have exluded some values in Q1
# similar to Q1, i would like to exclude outliers, such as players that never shot a free throw

wnba = wnba[(wnba["FTA"] > 0)]

print(wnba.shape)

(137, 32)


**Now that all the requirements have been taken into account, compute the confidence interval of the proportion with a confidence level of 95%:**

In [39]:
# your code here

# population is all female professional basketball players that have shot a free throw
# sub-population is all wnba players that have shot a free throw
p = (wnba["FTA"] > 0).count()

# 1. Set the hypothesis
# H0: mu <= 40 (the mean FT% is less (or equal) than 40%)
# H1: mu > 40 (the mean FT% is more than 40%)
mu = 40
cit = "greater"

# 2. Choose significance level
alpha = 0.05

# 3. Sample (take the whole sub-population)
n = 137
sample = wnba["FT%"].sample(n)

# 4. Compute the statistic & 5. Get p-value -> st.ttest_1samp
t_test_result = st.ttest_1samp(sample, mu, alternative = cit)

# 6. Decide (for "greater" choose ">", for "less" choose "<")
if t_test_result.pvalue < alpha and t_test_result.statistic > 0: 
    print("We can reject the null hypothesis")
else:
    print("We can not reject the null hypothesis")
    
# Give results
print()
print("population:", p)
print("hypothesis:", mu)
print("confidence interval tail:", cit)
print("significance level:", alpha)
print()
print("sample population:", n)
print("sample mean:", sample.mean())
print("sample std:", sample.std(ddof = 1))
print()
print("statistic:", t_test_result.statistic)
print("p-value:", t_test_result.pvalue)
print("df:", t_test_result.df)

We can reject the null hypothesis

population: 137
hypothesis: 40
confidence interval tail: greater
significance level: 0.05

sample population: 137
sample mean: 78.5963503649635
sample std: 11.709306308628321

statistic: 38.581166703103584
p-value: 2.0120042321359712e-75
df: 136


**What can you comment about our result? What would you tell your sister?**

In [15]:
#your-answer-here

# with 95% confidence I can say that the average of free throws is more than 40%
# under the definition of failure I gave (40%) most female players do not fail their free throws

# Bonus: Can you plot the probability distribution of the proportion of missed free throws, indicating where the critical region is?

In [ ]:
#your code here

# Question 3: Is the average number of assists for WNBA players only higher than the average for WNBA and NBA players together?

Your brother-in-law is convinced that the average assists for female professional players is higher than the average of both female and male players combined (which is 52 for the 2016-2017 season). You would like to actually prove if this is true or not but you remember your stats teacher saying "you can't *prove* anything, you just can say that *you are not* saying foolishness".

**How would you do it? Try and think about the requirements that your sample must satisfy in order to do that. Do you feel it actually fulfills those requirements? Do you need to make any assumptions?**

In [16]:
#your-answer-here

# There is the column AST which counts assists. 
So I can check, if 52 is an average that would be reached with only the 
# WNBA players, and if that is not true,
if only WNBA players are probable to show a higher average of assists.

**Use a two-tailed one-sample t-test to see if we can reject (or not) the null hypothesis with a 95% confidence level.**

In [41]:
#your code here

p = (wnba["AST"]).count()

# 1. Set the hypothesis
# H0: mu = 52
# H1: mu != 52
mu = 52
cit = "two-sided"

# 2. Choose significance level
alpha = 0.05

# 3. Sample
n = 142
sample = wnba["AST"].sample(n)

# 4. Compute the statistic & 5. Get p-value -> st.ttest_1samp
t_test_result = st.ttest_1samp(sample, mu, alternative = cit)

# 6. Decide
if t_test_result.pvalue < alpha: 
    print("We can reject the null hypothesis")
else:
    print("We can not reject the null hypothesis")
    
print()
print("population:", p)
print("hypothesis:", mu)
print("confidence interval tail:", cit)
print("significance level:", alpha)
print()
print("sample population:", n)
print("sample mean:", sample.mean())
print("sample std:", sample.std(ddof = 1))
print()
print("statistic:", t_test_result.statistic)
print("p-value:", t_test_result.pvalue)
print("df:", t_test_result.df)

We can reject the null hypothesis

population: 142
hypothesis: 52
confidence interval tail: two-sided
significance level: 0.05

sample population: 142
sample mean: 44.514084507042256
sample std: 41.49078952999805

statistic: -2.1499947192482898
p-value: 0.033261541354107166
df: 141


In [18]:
#your-answer-here

# The average number of assists for WNBA players is not 52

**Now use a one-tailed one-sample t-test to see if we can reject (or not) the null hypothesis with a 95% confidence level.**

In [42]:
#your-answer-here


p = (wnba["AST"]).count()

# 1. Set the hypothesis
# H0: mu <= 52
# H1: mu > 52
mu = 52
cit = "greater"

# 2. Choose significance level
alpha = 0.05

# 3. Sample
n = 142
sample = wnba["AST"].sample(n)

# 4. Compute the statistic & 5. Get p-value -> st.ttest_1samp
t_test_result = st.ttest_1samp(sample, mu, alternative = cit)

# 6. Decide
if t_test_result.pvalue < alpha and t_test_result.statistic > 0:  
    print("We can reject the null hypothesis")
else:
    print("We can not reject the null hypothesis")
    
print()
print("population:", p)
print("hypothesis:", mu)
print("confidence interval tail:", cit)
print("significance level:", alpha)
print()
print("sample population:", n)
print("sample mean:", sample.mean())
print("sample std:", sample.std(ddof = 1))
print()
print("statistic:", t_test_result.statistic)
print("p-value:", t_test_result.pvalue)
print("df:", t_test_result.df)

We can not reject the null hypothesis

population: 142
hypothesis: 52
confidence interval tail: greater
significance level: 0.05

sample population: 142
sample mean: 44.514084507042256
sample std: 41.49078952999803

statistic: -2.14999471924829
p-value: 0.9833692293229465
df: 141


In [ ]:
# The average number of assists for WNBA players is less than 52

# Bonus: Can you plot the resulting t-distribution of both tests? Indicate where the is the critical region and where does your statistic fall.**

In [ ]:
#your code here

# Bonus: Satisfying your curiosity

You finally managed to solve your family's debates over basketball! While you were doing that you started to take an interest in the normal distribution.

You read that the normal distribution is present in a lot of natural phenomenons, like blood pressure, IQ, weight and height. If, for example, we could plot the distribution of the weights of every human on the planet right now it would have the shape of a normal distribution.

In light of this you would like to see if it's possible to check if the distribution of the weights of the WNBA players is a sample distribution that comes from a population that has a normal distribution, because theoretically this should be the case.

**How would you try to demonstrate that our sample fits a normal distribution? What kind of test would you use? Would you have to make any assumptions?**

In [22]:
#your-answer-here

In [19]:
# your code here

**What are your comments in regards to the results of the test?**

In [24]:
#your-answer-here